In [481]:
import panel as pn
import folium as fm
import psycopg
from geopy.geocoders import Nominatim

In [482]:
height_widgets = 50

In [483]:
conn = psycopg.connect(dbname='fornecer', user='postgres', password='2703705')
cur = conn.cursor()
cur.execute("set schema 'fornecer';")
cur.execute('SELECT * FROM cliente;')
result = cur.fetchall()

In [484]:
button_login = pn.widgets.Button(name='ENTRAR', button_type='primary', height=height_widgets)
button_registration = pn.widgets.Button(name='Registre-se', height=height_widgets)

button_registration_client = pn.widgets.Button(name='Cadastrar', button_type='primary', height=height_widgets, width=620)
button_registration_company = pn.widgets.Button(name='Cadastrar', button_type='primary', height=height_widgets, width=620)
button_registration_unsubscribe = pn.widgets.Button(name='Cancelar cadastro', button_type='success', height=height_widgets, width=620)

button_search_company = pn.widgets.Button(name='BUSCAR', button_type='primary', height=height_widgets)
button_search_product = pn.widgets.Button(name='BUSCAR', button_type='primary', height=height_widgets)

button_exit = pn.widgets.Button(name='SAIR', button_type='success', height=height_widgets, width=100)

In [485]:
layout_main = pn.Column()
layout_sidebar = pn.Column()

In [486]:
template = pn.template.FastListTemplate(
    site="Fornecer",
    title="Login",
    main=[layout_main],
    sidebar=[],
    main_max_width='385px'
)

In [487]:
def start():
    screen_login('start')

def screen_login(event):
    for x in layout_main:
        layout_main.pop(len(layout_main)-1)
    
    if len(template.header) > 0:
        template.header.pop(0)
    
    template.main_max_width = '385px'
    template.title='Login'

    if len(template.sidebar) > 0:
        template.sidebar.pop(0)
    
    layout_main.append(pn.Row('<h2>Faça seu login</h2>'))
    layout_main.append(pn.Row(pn.widgets.TextInput(placeholder='Cpf/Cnpj', height=height_widgets)))
    layout_main.append(pn.Row(pn.widgets.PasswordInput(placeholder='Senha', height=height_widgets)))
    layout_main.append(pn.Row(button_login))
    layout_main.append(pn.Row('Ainda não tem uma conta?'))
    layout_main.append(pn.Row(button_registration))

def screen_register(event):
    for x in layout_main:
        layout_main.pop(len(layout_main)-1)
    
    template.main_max_width = '700px'
    template.title='Cadastro'

    register_client = pn.Column(
        pn.Row('<h3>Dados do cliente</h3><hr>'),
        pn.Row(pn.widgets.TextInput(placeholder='Cpf', height=height_widgets), pn.widgets.TextInput(placeholder='Email', height=height_widgets)),
        pn.Row(pn.widgets.TextInput(placeholder='Nome', height=height_widgets), pn.widgets.TextInput(placeholder='Sobrenome', height=height_widgets)),
        pn.Row(pn.widgets.PasswordInput(placeholder='Senha', height=height_widgets)),
        pn.Row('<h3>Localização</h3><hr>'),
        pn.Row(pn.widgets.TextInput(placeholder='Cep', height=height_widgets), pn.Row(pn.widgets.TextInput(placeholder='Região', height=height_widgets))),
        pn.Row(pn.widgets.TextInput(placeholder='Estado', height=height_widgets), pn.Row(pn.widgets.TextInput(placeholder='Cidade', height=height_widgets))),
        pn.Row(button_registration_client)
    )

    register_company = pn.Column(
        pn.Row('<h3>Dados da empresa</h3><hr>'),
        pn.Row(pn.widgets.TextInput(placeholder='Cnpj', height=height_widgets), pn.widgets.TextInput(placeholder='Email', height=height_widgets)),
        pn.Row(pn.widgets.TextInput(placeholder='Nome da empresa', height=height_widgets), pn.widgets.TextInput(placeholder='Mercado de atuação', height=height_widgets)),
        pn.Row(pn.widgets.PasswordInput(placeholder='Senha', height=height_widgets)),
        pn.Row('<h3>Localização</h3><hr>'),
        pn.Row(pn.widgets.TextInput(placeholder='Cep', height=height_widgets), pn.Row(pn.widgets.TextInput(placeholder='Região', height=height_widgets))),
        pn.Row(pn.widgets.TextInput(placeholder='Estado', height=height_widgets), pn.Row(pn.widgets.TextInput(placeholder='Cidade', height=height_widgets))),
        pn.Row(button_registration_company)
    )

    tab_registration = pn.Tabs(
        ('Cliente', register_client),
        ('Empresa', register_company)
    )
    
    layout_main.append(pn.Row('<h2>Crie sua conta</h2>'))
    layout_main.append(tab_registration)
    layout_main.append(button_registration_unsubscribe)

def screen_home_client(client_info):
    for x in layout_main:
        layout_main.pop(len(layout_main)-1)

    for x in layout_sidebar:
        layout_sidebar.pop(len(layout_sidebar)-1)
    
    template.header.append(pn.Row(button_exit))
    template.main_max_width=''
    template.title='Cliente'
    template.sidebar.append(layout_sidebar)

    map = fm.Map(location=[40.713,-74.006], zoom_start=5)

    tab_home_main_client = pn.Tabs(
        ('Mapa', pn.panel(map, height=570, width=1120)),
        ('Tabela', '<h3>Tabela</h3>')
    )

    search_company = pn.Column(
        pn.Row('<h3>Buscar por empresa</h3>'),
        pn.Row(pn.widgets.TextInput(placeholder='Nome da empresa')),
        pn.Row(pn.widgets.MultiChoice(placeholder='Tipo', options=['Alimento', 'Combustível', 'Manufatura'], solid=True)),
        pn.Row(pn.widgets.TextInput(placeholder='Região')),
        pn.Row(pn.widgets.TextInput(placeholder='Estado')),
        pn.Row(pn.widgets.TextInput(placeholder='Cidade')),
        pn.Row(button_search_company)
    )

    search_product = pn.Column(
        pn.Row('<h3>Buscar por produto</h3>'),
        pn.Row(pn.widgets.TextInput(placeholder='Nome do produto')),
        pn.Row(pn.widgets.MultiChoice(name='Tipo', options=['Alimento', 'Combustível', 'Manufatura'])),
        pn.Row(pn.widgets.RangeSlider(name='Intervalo de preço', start=0, end=1000, step=0.01)),
        pn.Row(button_search_product)
    )

    tab_home_sidebar_client = pn.Tabs(
        ('Empresa', search_company),
        ('Produto', search_product)
    )

    layout_main.append(pn.Row(tab_home_main_client))

    layout_sidebar.append(pn.Row(tab_home_sidebar_client))

def screen_home_company(company_info):
    for x in layout_main:
        layout_main.pop(len(layout_main)-1)

    for x in layout_sidebar:
        layout_sidebar.pop(len(layout_sidebar)-1)
    
    template.header.append(pn.Row(button_exit))
    template.main_max_width=''
    template.title='Empresa'
    template.sidebar.append(layout_sidebar)

    map = fm.Map(location=[49.896,-97.138], zoom_start=5)

    tab_home_main_client = pn.Tabs(
        ('Mapa', pn.panel(map, height=570, width=1120)),
        ('Tabela', '<h3>Tabela</h3>')
    )

    search_company = pn.Column(
        pn.Row('<h3>Buscar por empresa</h3>'),
        pn.Row(pn.widgets.TextInput(placeholder='Nome da empresa')),
        pn.Row(pn.widgets.MultiChoice(placeholder='Tipo', options=['Alimento', 'Combustível', 'Manufatura'], solid=True)),
        pn.Row(pn.widgets.TextInput(placeholder='Região')),
        pn.Row(pn.widgets.TextInput(placeholder='Estado')),
        pn.Row(pn.widgets.TextInput(placeholder='Cidade')),
        pn.Row(button_search_company)
    )

    search_product = pn.Column(
        pn.Row('<h3>Buscar por produto</h3>'),
        pn.Row(pn.widgets.TextInput(placeholder='Nome do produto')),
        pn.Row(pn.widgets.MultiChoice(name='Tipo', options=['Alimento', 'Combustível', 'Manufatura'])),
        pn.Row(pn.widgets.RangeSlider(name='Intervalo de preço', start=0, end=1000, step=0.01)),
        pn.Row(button_search_product)
    )

    tab_home_sidebar_client = pn.Tabs(
        ('Empresa', search_company),
        ('Produto', search_product)
    )

    layout_main.append(pn.Row(tab_home_main_client))

    layout_sidebar.append(pn.Row(tab_home_sidebar_client))

In [488]:
def check_login_client():
    cur.execute('SELECT cod_cliente, senha FROM cliente;')
    clients = cur.fetchall()

    login_client = None

    for login in clients:
        cpf = login[0]
        password = login[1]
        layout_main_cpf = layout_main[1][0].value
        layout_main_senha = layout_main[2][0].value
        if str(layout_main_cpf) == str(cpf) and str(layout_main_senha) == str(password):
            login_client = [str(cpf), str(password)]
            break
    
    return login_client

def check_login_company():
    cur.execute('SELECT cnpj, senha FROM fornecedor;')
    companies = cur.fetchall()

    login_company = None

    for login in companies:
        cnpj = login[0]
        password = login[1]
        layout_main_cnpj = layout_main[1][0].value
        layout_main_senha = layout_main[2][0].value
        if str(layout_main_cnpj) == str(cnpj) and str(layout_main_senha) == str(password):
            login_company = [str(cnpj), str(password)]
            break
    
    return login_company

def check_account(event):
    login_client = check_login_client()
    login_company = None

    if login_client is None:
        login_company = check_login_company()
    
    if login_client is None and login_company is None:
        if len(layout_main) == 7:
            layout_main.pop(6)
        layout_main.append(pn.Row('<p style="color:red">Login ou senha inválidos</p>'))
    else:
        if login_client is not None:
            cur.execute('SELECT * FROM cliente WHERE cod_cliente = ' + login_client[0])
            client_info = cur.fetchall()
            screen_home_client(client_info)
        else:
            cur.execute('SELECT * FROM fornecedor WHERE cnpj = ' + login_company[0])
            company_info = cur.fetchall()
            screen_home_company(company_info)

def register_client(event):
    cpf = layout_main[1][0][1][0]
    email = layout_main[1][0][1][1]
    nome = layout_main[1][0][2][0]
    sobrenome = layout_main[1][0][2][1]
    senha = layout_main[1][0][3][0]

    cep = layout_main[1][0][5][0]
    regiao = layout_main[1][0][5][1][0]
    estado = layout_main[1][0][6][0]
    cidade = layout_main[1][0][6][1][0]

    clients_data = [cpf, nome, sobrenome, cep, email, senha]
    location_data = [cep, regiao, estado, cidade]

    empty_values_clients = False
    empty_values_location = False

    for data in clients_data:
        if data.value == '':
            empty_values_clients = True
            break
    
    for data in location_data:
        if data.value == '':
            empty_values_location = True
            break
    
    if empty_values_clients or empty_values_location:
        if len(layout_main) > 3:
            layout_main.pop(3)
        layout_main.append(pn.Row('<p style="color:red">Preencha todos os dados</p>'))
    else:
        nome_completo = clients_data[1].value + " " + clients_data[2].value
        cur.execute("INSERT INTO cliente(cod_cliente, nome, cep_endereco, email, senha) VALUES (" +
            clients_data[0].value + ", " + 
            "'"+nome_completo+"'" + ", " +
            clients_data[3].value + ", " +
            "'"+clients_data[4].value+"'" + ", " +
            "'"+clients_data[5].value+"'" + ");"
        )
        conn.commit()

        loc = Nominatim(user_agent="fornecer")
        location = loc.geocode(cidade.value)

        lat = round(location.latitude, 3)
        lon = round(location.longitude, 3)

        cur.execute('INSERT INTO endereco(cep, regiao, estado, cidade, lat, lon) VALUES (' +
            location_data[0].value + ', ' +
            "'"+location_data[1].value+"'" + ', ' +
            "'"+location_data[2].value+"'" + ', ' +
            "'"+location_data[3].value+"'" + ', ' +
            str(lat) + ', ' +
            str(lon) + ');'
        )
        conn.commit()

        cur.execute('SELECT * FROM cliente WHERE cod_cliente = ' + cpf.value)
        client_info = cur.fetchall()
        screen_home_client(client_info)

def register_company(event):
    cnpj = layout_main[1][1][1][0]
    email = layout_main[1][1][1][1]
    nome = layout_main[1][1][2][0]
    mercado = layout_main[1][1][2][1]
    senha = layout_main[1][1][3][0]

    cep = layout_main[1][1][5][0]
    regiao = layout_main[1][1][5][1][0]
    estado = layout_main[1][1][6][0]
    cidade = layout_main[1][1][6][1][0]

    company_data = [cnpj, nome, mercado, cep, email, senha]
    location_data = [cep, regiao, estado, cidade]

    empty_values_clients = False
    empty_values_location = False

    for data in company_data:
        if data.value == '':
            empty_values_clients = True
            break
    
    for data in location_data:
        if data.value == '':
            empty_values_location = True
            break
    
    if empty_values_clients or empty_values_location:
        if len(layout_main) > 3:
            layout_main.pop(3)
        layout_main.append(pn.Row('<p style="color:red">Preencha todos os dados</p>'))
    else:
        cur.execute("INSERT INTO fornecedor(cnpj, nome, mercado_atuacao, cep_endereco, email, senha) VALUES (" +
            company_data[0].value + ", " + 
            "'"+company_data[1].value+"'" + ", " +
            "'"+company_data[2].value+"'" + ", " +
            company_data[3].value + ", " +
            "'"+company_data[4].value+"'" + ", "
            "'"+company_data[5].value+"'" + ");"
        )
        conn.commit()

        loc = Nominatim(user_agent="fornecer") 
        location = loc.geocode(cidade.value)

        lat = round(location.latitude, 3)
        lon = round(location.longitude, 3)
        
        cur.execute('INSERT INTO endereco(cep, regiao, estado, cidade, lat, lon) VALUES (' +
            location_data[0].value + ', ' +
            "'"+location_data[1].value+"'" + ', ' +
            "'"+location_data[2].value+"'" + ', ' +
            "'"+location_data[3].value+"'" + ', ' +
            str(lat) + ', ' +
            str(lon) + ');'
        )
        conn.commit()

        cur.execute('SELECT * FROM fornecedor WHERE cnpj = ' + cnpj.value)
        company_info = cur.fetchall()
        screen_home_company(company_info)

In [489]:
button_registration.param.watch(screen_register, 'value')
button_registration.value = False

button_registration_unsubscribe.param.watch(screen_login, 'value')
button_registration_unsubscribe.value = False

button_login.param.watch(check_account, 'value')
button_login.value = False

button_exit.param.watch(screen_login, 'value')
button_exit.value = False

button_registration_client.param.watch(register_client, 'value')
button_registration_client.value = False

button_registration_company.param.watch(register_company, 'value')
button_registration_company.value = False

In [490]:
start()

template.show()

Launching server at http://localhost:51347
